In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import sys
from PIL import Image
import pickle
import h5py
import cv2
from skimage.transform import resize

sys.path.append(os.path.dirname(os.path.abspath('')))
from utils.directory import DATA_DIR, DATASET_DIR
import utils.prepare_data as pd
import utils.error as error
import utils.visualize as visual
import utils.convert_xyz_uvd as xyzuvd
# jupyter nbconvert --to html evaluation.ipynb

In [ ]:
epoch = 100
exp = 'pose_net_aug'

data_split = 'train'
save_prefix = 'train_fpha'
keys_cache_file = os.path.join(DATA_DIR, save_prefix + '_keys_cache.p')
train_keys = pickle.load(open(keys_cache_file, "rb"))
dataroot_uvd_gt_scaled = os.path.join(DATA_DIR, save_prefix + '_uvd_gt_scaled.lmdb')
dataroot_xyz_gt_saved = os.path.join(DATA_DIR, save_prefix + '_xyz_gt.lmdb')

train_uvd_gt_scaled = pd.read_all_lmdb_from_file(train_keys, dataroot_uvd_gt_scaled, np.float32, (21, 3))
train_xyz_gt = pd.read_all_lmdb_from_file(train_keys, dataroot_xyz_gt_saved, np.float32, (21, 3))
train_uvd_gt = xyzuvd.xyz2uvd_color(np.asarray(train_xyz_gt))

pred_file = os.path.join(DATA_DIR, exp, 'predict_%s_%s.txt' %(epoch, data_split))
train_pred_uvd = pd.get_pred_uvd(pred_file, save_prefix)

f= h5py.File(os.path.join(DATA_DIR, exp, 'scoremap_%s_%s.h5' %(epoch, data_split)))
train_scoremap = f['scoremap'][...]
f.close()

data_split = 'test'
save_prefix = 'test_fpha'
keys_cache_file = os.path.join(DATA_DIR, save_prefix + '_keys_cache.p')
test_keys = pickle.load(open(keys_cache_file, "rb"))
dataroot_uvd_gt_scaled = os.path.join(DATA_DIR, save_prefix + '_uvd_gt_scaled.lmdb')
dataroot_xyz_gt_saved = os.path.join(DATA_DIR, save_prefix + '_xyz_gt.lmdb')

test_uvd_gt_scaled = pd.read_all_lmdb_from_file(test_keys, dataroot_uvd_gt_scaled, np.float32, (21, 3))
test_xyz_gt = pd.read_all_lmdb_from_file(test_keys, dataroot_xyz_gt_saved, np.float32, (21, 3))
test_uvd_gt = xyzuvd.xyz2uvd_color(np.asarray(test_xyz_gt))

f= h5py.File(os.path.join(DATA_DIR, exp, 'scoremap_%s_%s.h5' %(epoch, data_split)))
test_scoremap = f['scoremap'][...]
f.close()

pred_file = os.path.join(DATA_DIR, exp, 'predict_%s_%s.txt' %(epoch, data_split))
test_pred_uvd = pd.get_pred_uvd(pred_file, save_prefix)

train_uvd_gt = train_uvd_gt[:, :, :2]
test_uvd_gt = test_uvd_gt[:, :, :2]

In [ ]:
coord_frame = 'uvd'
pred = train_pred_uvd
true = train_uvd_gt
print('%s TRAIN 2D mean_l2_error: ' %coord_frame, error.mean_pose_error(true, pred))
pred = test_pred_uvd
true = test_uvd_gt
print('%s TEST 2D mean_l2_error: ' %coord_frame, error.mean_pose_error(true, pred))

In [ ]:
# Percentage of correct key points
# pred = np.reshape(train_pred_uvd, (-1, 42))
# true = np.asarray(train_uvd_gt)
pck = error.percentage_frames_within_error_curve_zimmmerman(true, pred)
print(pck)
thresholds = np.arange(0, 85, 5)
print('AUC:', error.calc_auc(pck, thresholds))

In [ ]:
# Percentage of correct key points
# pred = np.reshape(test_pred_uvd, (-1, 42))
# true = np.asarray(test_uvd_gt)
pck = error.percentage_frames_within_error_curve_zimmmerman(true, pred)
print('[', end =" ")
for num in pck:
    print('%f, ' %num, end =" ")
print(']')
thresholds = np.arange(0, 85, 5)
print('AUC:', error.calc_auc(pck, thresholds))

In [ ]:
idx = 0
file_name = os.path.join(DATASET_DIR, 'First_Person_Action_Benchmark', 'Video_files', train_keys[idx])
visual.show_true_pred_img_and_skel_color(file_name, train_uvd_gt[idx], train_pred_uvd[idx])

In [ ]:
file_name = os.path.join(DATASET_DIR, 'First_Person_Action_Benchmark', 'Video_files', test_keys[idx])
visual.show_true_pred_img_and_skel_color(file_name, test_uvd_gt[idx], test_pred_uvd[idx])

In [ ]:
# idx = 1
# img = Image.open()
# scoremap_gt = pd.create_multiple_gaussian_map(train_uvd_gt_scaled[idx], (256, 256))

# scoremap_i = pd.sk_resize(test_scoremap[idx], (256, 256))
# for i in range(21):
#     fig, ax = plt.subplots(1, 2)
#     ax[0].imshow(visual.blend_scoremap_img(scoremap_i[:, :, i], img))
#     ax[0].set_title('pred')
#     ax[1].imshow(visual.blend_scoremap_img(scoremap_gt[:, :, i], img))
#     ax[1].set_title('true')
    